In [1]:
!pip install pydantic-settings

In [2]:
pip install chromadb==0.3.21

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.6/442.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:

In [3]:
!pip install pydantic-settings
!pip install --upgrade chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:

In [35]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from chromadb.config import Settings
from chromadb import Client
model_name = "Salesforce/codet5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


csv_file ="/content/final_python_dataset.csv"
df = pd.read_csv(csv_file)
codes = df['text'].to_list()
ids = df['task_id'].to_list()


In [36]:
def generate_embedding(code):
    inputs = tokenizer(code, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        # Use only the encoder to generate embeddings
        encoder_outputs = model.encoder(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )
    # Take the mean of the encoder's last hidden states
    return encoder_outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [37]:
dp_path = "/content/python_codet5_disc_Vdb"
import os
import chromadb
os.makedirs(dp_path , exist_ok= True)
client = chromadb.PersistentClient(path=dp_path)
collection = client.get_or_create_collection(name="python_codet5_disc_collection")

for i in range (len(codes)):
    embedding = generate_embedding(codes[i])
    print(f"Code {i}: Embedding size = {embedding.shape}")
    collection.add(
        documents=[codes[i]],
        metadatas=[{"index": str(ids[i])}],
        ids=[str(ids[i])],
        embeddings=[embedding.tolist()]
    )

print(collection.get()['documents'])

Code 0: Embedding size = (768,)
Code 1: Embedding size = (768,)
Code 2: Embedding size = (768,)
Code 3: Embedding size = (768,)
Code 4: Embedding size = (768,)
Code 5: Embedding size = (768,)
Code 6: Embedding size = (768,)
Code 7: Embedding size = (768,)
Code 8: Embedding size = (768,)
Code 9: Embedding size = (768,)
Code 10: Embedding size = (768,)
Code 11: Embedding size = (768,)
Code 12: Embedding size = (768,)
Code 13: Embedding size = (768,)
Code 14: Embedding size = (768,)
Code 15: Embedding size = (768,)
Code 16: Embedding size = (768,)
Code 17: Embedding size = (768,)
Code 18: Embedding size = (768,)
Code 19: Embedding size = (768,)
Code 20: Embedding size = (768,)
Code 21: Embedding size = (768,)
Code 22: Embedding size = (768,)
Code 23: Embedding size = (768,)
Code 24: Embedding size = (768,)
Code 25: Embedding size = (768,)
Code 26: Embedding size = (768,)
Code 27: Embedding size = (768,)
Code 28: Embedding size = (768,)
Code 29: Embedding size = (768,)
Code 30: Embedding s

In [38]:
!zip -r python_codet5_disc_Vdb.zip /content/python_codet5_disc_Vdb
from google.colab import files
files.download("python_codet5_disc_Vdb.zip")

  adding: content/python_codet5_disc_Vdb/ (stored 0%)
  adding: content/python_codet5_disc_Vdb/6a54b667-2c15-4ddb-a9a1-fed743ad30e2/ (stored 0%)
  adding: content/python_codet5_disc_Vdb/6a54b667-2c15-4ddb-a9a1-fed743ad30e2/index_metadata.pickle (deflated 48%)
  adding: content/python_codet5_disc_Vdb/6a54b667-2c15-4ddb-a9a1-fed743ad30e2/link_lists.bin (deflated 87%)
  adding: content/python_codet5_disc_Vdb/6a54b667-2c15-4ddb-a9a1-fed743ad30e2/header.bin (deflated 60%)
  adding: content/python_codet5_disc_Vdb/6a54b667-2c15-4ddb-a9a1-fed743ad30e2/data_level0.bin (deflated 10%)
  adding: content/python_codet5_disc_Vdb/6a54b667-2c15-4ddb-a9a1-fed743ad30e2/length.bin (deflated 95%)
  adding: content/python_codet5_disc_Vdb/chroma.sqlite3 (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>